## CodeI/O

Original paper (DeepSeek): https://arxiv.org/pdf/2502.07316

The approach begins by obtaining high quality raw code data and preprocessing it by prompting an LLM. The output of this preprocessing, for each raw code file used, should be:

- cleaned reference code, with a main entrypoint function
- a query, converting the reference code into a question (along the lines of "given [function parameters...] how can we obtain [desired outputs...]")
- a natural language description of all inputs (function parameters) and outputs (function return values)
- an input generator, which can generate a dictionary of valid inputs for the function

This notebook seeks to experiment with prompting an LLM to this end, as a starting point. The raw code data is from this GitHub repository that the DeepSeek paper mentions as one of their raw code sources: https://github.com/TheAlgorithms/Python

NOTE: Be careful with the raw code you input into this, as cells later execute the LLM-generated outputs.

In [1]:
import random
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
raw_files = list(Path("raw_files/").iterdir())

Note that the below prompt is built for DeepSeekV3. It may not work with other LLMs.

In [9]:
format_prompt_template = """
You are tasked with preprocessing a raw file of Python code into a standard format. The format is made up of several components. Here is a very simple example of a raw code file:

def kg_to_pounds(weights):
    return [w * 2.20462 for w in weights]

def filter_weekly(original_measurements, days):
    return [m for i, m in enumerate(original_measurements) if i % 7 == 0]

def main(kgs, days):
    lbs = kg_to_pounds(kgs)

    for measurement in filter_weekly(lbs, days):
        print(measurement)

1. Cleaned reference code, with a main entrypoint function that takes all required arguments as parameters and returns all outputs.

The name of the main entrypoint function should be `main`. The parameters should be clearly named but do not require type hints. The function should return a dict mapping output names to values. The function should contain all the necessary code to perform the functionality, without splitting into several functions. The function should not print or otherwise output anything; results should be returned as part of the result dict. Ensure you include any imports necessary, prior to the function definition.

Example function signature: `def main(weights_kg, days):`

2. A query, defined as natural language description of the question the function answers.

Example query: "You are given two lists of integers, `weights_kg` and `days`. The unit of `weights_kg` is kilograms. `days` refers to the number of days passed, starting from zero. Your task is to convert the integers to pounds and filter to only one weight measurement every 7 days. Return the list of integers in pounds."

The query should be as detailed as the code requires to be fully explained. It should be clear what the function does, what the inputs are, and what the outputs are.

3. A natural language description of all inputs (function parameters) and outputs (return values) of the function.

Example description:

Input:
    weights_kg (list of int): List of weight values in kilograms.
    days (list of int): List of integers representing the number of days passed, starting from zero.

Output:
    return (dict): A dictionary with one key:
    - weights_lb (list of int): List of filtered weight values in pounds.

4. Python 3.11 code for an input generator, which randomly generates valid sets of inputs for the functions.

The input generator should return a dict mapping parameter names to values. The values should be randomly generated, but should be valid inputs for the function.

Example input generator:

def input_generator():
    weights = [np.random.uniform(0, 100) for _ in range(40)]
    days = list(range(40))
    return {{"weights_kg": weights, "days": days}}

Using the guidelines and example above, preprocess the following raw code file into the standard format:

{0}

Output the components (reference code, query, description, input generator) in order. Separate each component with a line of dashes (---). Avoid code blocks and do not output any Markdown formatting. Respond only with the four components, no prefix or additional text.
"""

In [3]:
import os
import time
from openai import OpenAI
from openai.types.chat import ChatCompletion, ChatCompletionMessageParam
from typing import Any, Iterable

def llm_generate(
    client: OpenAI,
    messages: Iterable[ChatCompletionMessageParam],
    sampling_params: dict[str, Any],
) -> ChatCompletion:
    max_retry = 3
    for trial in range(max_retry):
        try:
            return client.chat.completions.create(
                messages=messages,
                **sampling_params,
            )
        except Exception as e:
            print("failure response:", e)
            time.sleep(trial * trial)  # quadratic backoff
            if trial == max_retry - 1:
                raise

open_router_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    timeout=90.0,
)

sampling_params = {
    "model": "deepseek/deepseek-chat:free",
    "max_tokens": 8192,
}

In [12]:
raw_file = random.choice(raw_files)

print(raw_file)

raw_code = raw_file.read_text()

prompt = format_prompt_template.format(raw_code)

messages = [
    {"role": "user", "content": prompt},
]

response = llm_generate(open_router_client, messages, sampling_params)
print(response.choices[0].message.content)

raw_files/climbing_stairs.py
def main(number_of_steps):
    assert isinstance(number_of_steps, int) and number_of_steps > 0, (
        f"number_of_steps needs to be positive integer, your input {number_of_steps}"
    )
    if number_of_steps == 1:
        return {"distinct_ways": 1}
    previous, current = 1, 1
    for _ in range(number_of_steps - 1):
        current, previous = current + previous, current
    return {"distinct_ways": current}

---
You are given an integer `number_of_steps` representing the number of steps on a staircase. Your task is to calculate the number of distinct ways to climb the staircase, where each time you can either climb 1 or 2 steps. Return the number of distinct ways as an integer.

---
Input:
    number_of_steps (int): The number of steps on the staircase. Must be a positive integer.

Output:
    return (dict): A dictionary with one key:
    - distinct_ways (int): The number of distinct ways to climb the staircase.

---
def input_generator():
    impor

In [13]:
code, query, parameters, generator = response.choices[0].message.content.split("\n---\n")

The below cell executes arbitrary code, so be careful with what you run.

In [14]:
def generate_io_pairs(main_code: str, input_generator_code: str, num_pairs: int = 100):
    local_vars = {}
    exec(main_code, {}, local_vars)
    exec(input_generator_code, {}, local_vars)
    io_pairs = []
    for _ in range(num_pairs):
        inputs = local_vars["input_generator"]()
        outputs = local_vars["main"](**inputs)
        io_pairs.append((inputs, outputs))
    return io_pairs

io_pairs = generate_io_pairs(code, generator, num_pairs=2)

In [15]:
io_pairs

[({'number_of_steps': 65}, {'distinct_ways': 27777890035288}),
 ({'number_of_steps': 19}, {'distinct_ways': 6765})]

Next we need to synthesize chains of thought from the LLM for use in building a supervised finetuning dataset. From the paper:

> Since we aim for the input-output prediction tasks, we construct the prompt using a designed template to combine the function, the query, the reference code, and either a specific input or output. The response should ideally be a natural language CoT to reason about how to derive the correct output or a feasible input.

The below prompts are from the paper.

In [16]:
synthetic_cot_prompt_prefix = """
You are given a question that requires some input and output variables as follows:

{0}

The input and output requirements are as follows:

{1}
"""

synthetic_cot_prompt_suffix = """
Tip: Here is a reference code snippet for this question. You can refer to this code to guide your reasoning but not copy spans of code directly.

{3}
"""

synthetic_cot_prompt_input_prediction = synthetic_cot_prompt_prefix + """
Given the following output:

{2}

Can you predict a feasible input without writing any code? Please reason and put your final answer in the following json format: "input": <your input>, where <your input> should be a dictionary, even if the there is only one input variable, with keys strictly matching the input variables' names as specified.
""" + synthetic_cot_prompt_suffix

synthetic_cot_prompt_output_prediction = synthetic_cot_prompt_prefix + """
Given the following input:

{2}

Can you predict the output without writing any code? Please reason and put your final answer in the following json format: "output": <your output>, where <your output> should strictly match the the output requirement as specified.
""" + synthetic_cot_prompt_suffix

In [17]:
def predict_input(query, parameters, output, reference_code):
    messages = [
        {"role": "user", "content": synthetic_cot_prompt_input_prediction.format(query, parameters, output, reference_code)},
    ]
    response = llm_generate(open_router_client, messages, sampling_params)
    return response.choices[0].message.content

predict_input(query, parameters, io_pairs[0][1], code)

'To determine the input `number_of_steps` that results in the output `{\'distinct_ways\': 27777890035288}`, we need to understand that this problem is related to the Fibonacci sequence. Specifically, the number of distinct ways to climb `n` steps, where you can climb either 1 or 2 steps at a time, is equal to the `(n+1)`-th Fibonacci number.\n\nGiven the output `27777890035288`, we need to find the integer `n` such that the `(n+1)`-th Fibonacci number is `27777890035288`.\n\nThe Fibonacci sequence grows exponentially, and the number `27777890035288` is a very large Fibonacci number. To find the corresponding `n`, we can use the fact that the Fibonacci sequence follows the recurrence relation:\n\n\\[ F(n) = F(n-1) + F(n-2) \\]\n\nGiven that `F(73) = 806515533049393` and `F(72) = 498454011879264`, it is clear that `27777890035288` is much smaller than `F(73)`. We need to find the exact `n` such that `F(n+1) = 27777890035288`.\n\nHowever, calculating Fibonacci numbers manually for large `

In [18]:
def predict_output(query, parameters, input, reference_code):
    messages = [
        {"role": "user", "content": synthetic_cot_prompt_output_prediction.format(query, parameters, input, reference_code)},
    ]
    response = llm_generate(open_router_client, messages, sampling_params)
    return response.choices[0].message.content

predict_output(query, parameters, io_pairs[1][0], code)

'To solve this problem, we need to calculate the number of distinct ways to climb a staircase with `number_of_steps` steps, where you can either take 1 or 2 steps at a time. This problem is a classic example of a dynamic programming problem and is very similar to the Fibonacci sequence.\n\n### Reasoning:\n- The number of distinct ways to climb `n` steps is equal to the sum of the number of distinct ways to climb `n-1` steps and the number of distinct ways to climb `n-2` steps. This is because from the `n-1`th step, you can take a single step to reach the `n`th step, and from the `n-2`th step, you can take two steps to reach the `n`th step.\n- The base cases are:\n  - For `n = 1`, there is only 1 way to climb the staircase (taking a single step).\n  - For `n = 2`, there are 2 ways to climb the staircase (taking two single steps or one double step).\n\nThe number of distinct ways to climb `n` steps follows the Fibonacci sequence. The Fibonacci sequence is defined as follows:\n- F(0) = 0\